In [ ]:
#IDEE clustering (e forse classificazione):
#Giocatori più adatti a determinata superficie (features punti durante gli anni su quella superficie, vittorie su superficie)

#Giocatori più adatti ai quattro Grandi Slam 

#Giocatori più adatti in base al livello del torneo, in base a chi li gioca (250, 500, 1000, GS)

#Giocatori più adatti in base al continente (partite vinte rispetto a quelle giocate in quel continente)

#Giocatori più favoriti in base a torneo, continente, superficie







#Giocatori più adatti a determinati trimestri dell'anno (features punti durante gli anni su quella superficie, vittorie su superficie)